In [0]:
spark.conf.set("fs.azure.account.auth.type.demostorage31225 .dfs.core.windows.net", "SAS")
spark.conf.set("fs.azure.sas.token.provider.type.demostorage31225 .dfs.core.windows.net", "org.apache.hadoop.fs.azurebfs.sas.FixedSASTokenProvider")
spark.conf.set("fs.azure.sas.fixed.token.demostorage31225.dfs.core.windows.net", "sv=2024-11-04&ss=bfqt&srt=sco&sp=rwdlacupyx&se=2026-12-04T23:43:15Z&st=2025-12-04T15:28:15Z&spr=https&sig=p3uQSzMKIs1%2Bg7O5%2FIU5zycxNZ%2BXHeUqMQy8Ah4a0Qc%3D")

path="abfss://demtest@demostorage31225.dfs.core.windows.net/bronze"
df1=spark.read.format("csv").option("header","true").option("inferSchema","True").load(path)
display(df1)

---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-5259901752555447>, line 6
      3 spark.conf.set("fs.azure.sas.fixed.token.demostorage31225.dfs.core.windows.net", "sv=2024-11-04&ss=bfqt&srt=sco&sp=rwdlacupyx&se=2026-12-04T23:43:15Z&st=2025-12-04T15:28:15Z&spr=https&sig=p3uQSzMKIs1%2Bg7O5%2FIU5zycxNZ%2BXHeUqMQy8Ah4a0Qc%3D")
      5 path="abfss://demtest@demostorage31225.dfs.core.windows.net/bronze"
----> 6 df1=spark.read.format("csv").option("header","true").option("inferSchema","True").load(path)
      7 display(df1)

File /databricks/spark/python/pyspark/instrumentation_utils.py:47, in _wrap_function.<locals>.wrapper(*args, **kwargs)
     45 start = time.perf_counter()
     46 try:
---> 47     res = func(*args, **kwargs)
     48     logger.log_success(
     49         module_name, class_name, function_name, time.perf_counter() - start, signature
     50     )
     51  

In [0]:
path = "abfss://demtest@demostorage31225.dfs.core.windows.net/bronze"
files_and_folders = dbutils.fs.ls(path)
display(files_and_folders)

In [0]:
spark.conf.set(
    "fs.azure.account.auth.type.demostorage31225.dfs.core.windows.net",
    "SAS"
)
spark.conf.set(
    "fs.azure.sas.token.provider.type.demostorage31225.dfs.core.windows.net",
    "org.apache.hadoop.fs.azurebfs.sas.FixedSASTokenProvider"
)
spark.conf.set(
    "fs.azure.sas.fixed.token.demostorage31225.dfs.core.windows.net",
    "sv=2024-11-04&ss=bfqt&srt=sco&sp=rwdlacupyx&se=2026-12-04T19:23:02Z&st=2025-12-04T11:08:02Z&spr=https&sig=Mze%2BtSwyvTpcu8utJiZSWBi1BvNyorHcqBK1kVcAn70%3D"
)

df = spark.read.format("csv").option("header", "true").load("abfss://demtest@demostorage31225.dfs.core.windows.net/people-100.csv")
display(df)

In [0]:
input_path = "abfss://demtest@demostorage31225.dfs.core.windows.net/people-100.csv"
df = spark.read.option("header", "true").csv(input_path)

# Write the CSV DataFrame as Parquet
output_path = "abfss://demtest@demostorage31225.dfs.core.windows.net/people-100.parquet"
df.write.mode("overwrite").option("compression", "snappy").parquet(output_path)

# Verify by reading the Parquet file back
df_parquet = spark.read.parquet(output_path)
display(df_parquet)


In [0]:

df_transactions = spark.read.parquet("abfss://demtest@demostorage31225.dfs.core.windows.net/bronze/transaction/")
df_products = spark.read.parquet("abfss://demtest@demostorage31225.dfs.core.windows.net/bronze/product/")
df_stores = spark.read.parquet("abfss://demtest@demostorage31225.dfs.core.windows.net/bronze/store/")
df_customers = spark.read.parquet("abfss://demtest@demostorage31225.dfs.core.windows.net/bronze/customer /pratikfiza/AzureProjectSource/refs/heads/main/customers.parquet")


# display(df_products)
# display(df_stores)
# display(df_customers)

In [0]:
display(df_transactions)

In [0]:
display(df_products)


In [0]:
display(df_stores)


In [0]:
display(df_customers)

In [0]:
# DBTITLE 1,create - 
from pyspark.sql.functions import col

# Convert types and clean data
df_transactions = df_transactions.select(
    col("transaction_id").cast("int"),
    col("customer_id").cast("int"),
    col("product_id").cast("int"),
    col("store_id").cast("int"),
    col("quantity").cast("int"),
    col("transaction_date").cast("date")
)

df_products = df_products.select(
    col("product_id").cast("int"),
    col("product_name"),
    col("category"),
    col("price").cast("double")
)

df_stores = df_stores.select(
    col("store_id").cast("int"),
    col("store_name"),
    col("location")
)

df_customers = df_customers.select(
    "customer_id", "first_name", "last_name", "email", "city", "registration_date"
).dropDuplicates(["customer_id"])





In [0]:
display(df_products)
display(df_stores)
display(df_customers)

In [0]:
# COMMAND ----------

# DBTITLE 1,join all data together
# Join all data
df_silver = df_transactions \
    .join(df_customers, "customer_id") \
    .join(df_products, "product_id") \
    .join(df_stores, "store_id") \
    .withColumn("total_amount", col("quantity") * col("price"))


# COMMAND ----------

display(df_silver)

In [0]:

# COMMAND ----------

# DBTITLE 1,dump to adls location
silver_path = "abfss://demtest@demostorage31225.dfs.core.windows.net/silver"

df_silver.write.mode("overwrite").format("delta").save(silver_path)



In [0]:
df_silver_read = spark.read.format("delta").load(silver_path)
display(df_silver_read)

store_id,product_id,customer_id,transaction_id,quantity,transaction_date,first_name,last_name,email,city,registration_date,product_name,category,price,store_name,location,total_amount
5,7,101,28,3,2024-11-15,Ravi,Yadav,user101@example.com,Delhi,2023-09-14,Smartwatch,Electronics,4999.0,Mega Plaza,Chennai,14997.0
3,1,102,11,2,2024-08-11,Nina,Joshi,user102@example.com,Mumbai,2024-01-21,Wireless Mouse,Electronics,799.99,Tech World Outlet,Bangalore,1599.98
4,1,103,18,3,2024-09-05,Sonal,Sharma,user103@example.com,Bangalore,2023-07-10,Wireless Mouse,Electronics,799.99,Downtown Mini Store,Pune,2399.9700000000003
3,3,104,13,4,2025-05-04,Karan,Patel,user104@example.com,Hyderabad,2024-02-05,Yoga Mat,Fitness,499.0,Tech World Outlet,Bangalore,1996.0
3,1,105,21,5,2024-10-02,Riya,Singh,user105@example.com,Chennai,2023-06-28,Wireless Mouse,Electronics,799.99,Tech World Outlet,Bangalore,3999.95
2,5,105,5,1,2025-03-17,Riya,Singh,user105@example.com,Chennai,2023-06-28,Notebook Set,Stationery,149.0,High Street Store,Delhi,149.0
4,3,105,2,5,2024-11-12,Riya,Singh,user105@example.com,Chennai,2023-06-28,Yoga Mat,Fitness,499.0,Downtown Mini Store,Pune,2495.0
3,9,107,22,4,2024-11-16,Priya,Kapoor,user107@example.com,Ahmedabad,2023-05-12,Dumbbell Set,Fitness,1999.0,Tech World Outlet,Bangalore,7996.0
1,5,108,12,4,2025-05-26,Rahul,Verma,user108@example.com,Kolkata,2023-08-19,Notebook Set,Stationery,149.0,City Mall Store,Mumbai,596.0
5,8,109,17,5,2024-07-10,Pooja,Mehta,user109@example.com,Delhi,2024-04-01,Desk Organizer,Accessories,399.0,Mega Plaza,Chennai,1995.0


In [0]:
# SCD1: Overwrite existing customer records with latest data (no history)
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number

# Assume df_customers contains the latest customer data
# SCD1: Upsert into Delta table (retail_silver_cleaned) by overwriting matching customer_id
from delta.tables import DeltaTable

delta_table = DeltaTable.forPath(spark, silver_path)

# Prepare the new data (df_silver)
updatesDF = df_silver

# SCD1: Merge (upsert) - overwrite matching records, insert new
delta_table.alias("target").merge(
    updatesDF.alias("source"),
    "target.customer_id = source.customer_id AND target.transaction_id = source.transaction_id"
).whenMatchedUpdateAll() \
 .whenNotMatchedInsertAll() \
 .execute()

# SCD2: Maintain full history of changes for customers (track changes over time)
# For demo, let's assume we want to track changes in customer city or email

from pyspark.sql.functions import lit, current_date

# Add SCD2 columns
df_scd2 = df_customers \
    .withColumn("start_date", current_date()) \
    .withColumn("end_date", lit(None).cast("date")) \
    .withColumn("is_current", lit(True))

# Write SCD2 table (overwrite for demo, use merge for production)
scd2_path = "abfss://demtest@demostorage31225.dfs.core.windows.net/silver"
# df_scd2.write.mode("overwrite").format("delta").save(scd2_path)
display(df_customers)
display(df_scd2)
# # Register SCD2 table
# spark.sql(f"""
# CREATE TABLE IF NOT EXISTS scd2_customers
# USING DELTA
# LOCATION '{scd2_path}'
# """)

# display(spark.read.format("delta").load(scd2_path))





customer_id,first_name,last_name,email,city,registration_date
101,Ravi,Yadav,user101@example.com,Delhi,2023-09-14
102,Nina,Joshi,user102@example.com,Mumbai,2024-01-21
103,Sonal,Sharma,user103@example.com,Bangalore,2023-07-10
104,Karan,Patel,user104@example.com,Hyderabad,2024-02-05
105,Riya,Singh,user105@example.com,Chennai,2023-06-28
106,Ajay,Mishra,user106@example.com,Pune,2024-03-10
107,Priya,Kapoor,user107@example.com,Ahmedabad,2023-05-12
108,Rahul,Verma,user108@example.com,Kolkata,2023-08-19
109,Pooja,Mehta,user109@example.com,Delhi,2024-04-01
110,Deepak,Nair,user110@example.com,Mumbai,2023-10-14


customer_id,first_name,last_name,email,city,registration_date,start_date,end_date,is_current
101,Ravi,Yadav,user101@example.com,Delhi,2023-09-14,2025-12-07,null,true
102,Nina,Joshi,user102@example.com,Mumbai,2024-01-21,2025-12-07,null,true
103,Sonal,Sharma,user103@example.com,Bangalore,2023-07-10,2025-12-07,null,true
104,Karan,Patel,user104@example.com,Hyderabad,2024-02-05,2025-12-07,null,true
105,Riya,Singh,user105@example.com,Chennai,2023-06-28,2025-12-07,null,true
106,Ajay,Mishra,user106@example.com,Pune,2024-03-10,2025-12-07,null,true
107,Priya,Kapoor,user107@example.com,Ahmedabad,2023-05-12,2025-12-07,null,true
108,Rahul,Verma,user108@example.com,Kolkata,2023-08-19,2025-12-07,null,true
109,Pooja,Mehta,user109@example.com,Delhi,2024-04-01,2025-12-07,null,true
110,Deepak,Nair,user110@example.com,Mumbai,2023-10-14,2025-12-07,null,true


In [0]:
# Gold Layer: Aggregate and business-level metrics

from pyspark.sql.functions import sum as _sum, countDistinct

# Example 1: Total sales per store, per product, per month
from pyspark.sql.functions import month, year

df_gold_sales = df_silver.groupBy(
    "store_id", "store_name", "product_id", "product_name", year("transaction_date").alias("year"), month("transaction_date").alias("month")
).agg(
    _sum("total_amount").alias("monthly_sales"),
    _sum("quantity").alias("monthly_quantity"),
    countDistinct("customer_id").alias("unique_customers")
)

# Example 2: Customer lifetime value
df_gold_customer_ltv = df_silver.groupBy(
    "customer_id", "first_name", "last_name", "email", "city"
).agg(
    _sum("total_amount").alias("customer_lifetime_value"),
    countDistinct("transaction_id").alias("total_transactions")
)

# Write gold tables to ADLS in Delta format
gold_sales_path = "abfss://demtest@demostorage31225.dfs.core.windows.net/gold/sales"
gold_customer_ltv_path = "abfss://demtest@demostorage31225.dfs.core.windows.net/gold/customer_ltv"

df_gold_sales.write.mode("overwrite").format("delta").save(gold_sales_path)
df_gold_customer_ltv.write.mode("overwrite").format("delta").save(gold_customer_ltv_path)

# Display gold layer DataFrames

display(df_gold_sales)
display(df_gold_customer_ltv)


from pyspark.sql.functions import sum as _sum, countDistinct, desc, to_date

# Total Sales Revenue
total_sales_revenue = df_silver.agg(_sum("total_amount").alias("total_sales_revenue"))
display(total_sales_revenue)

# Daily Sales Trends
daily_sales_trends = df_silver.groupBy(
    to_date("transaction_date").alias("date")
).agg(
    _sum("total_amount").alias("daily_sales"),
    _sum("quantity").alias("daily_quantity")
).orderBy("date")
display(daily_sales_trends)

# Monthly Sales Trends
monthly_sales_trends = df_silver.groupBy(
    year("transaction_date").alias("year"),
    month("transaction_date").alias("month")
).agg(
    _sum("total_amount").alias("monthly_sales"),
    _sum("quantity").alias("monthly_quantity")
).orderBy("year", "month")
display(monthly_sales_trends)

# Top Selling Products by Revenue
top_products = df_silver.groupBy(
    "product_id", "product_name"
).agg(
    _sum("total_amount").alias("product_revenue"),
    _sum("quantity").alias("total_quantity")
).orderBy(desc("product_revenue"))
display(top_products)

# Top Categories by Revenue
top_categories = df_silver.groupBy(
    "category"
).agg(
    _sum("total_amount").alias("category_revenue")
).orderBy(desc("category_revenue"))
display(top_categories)

# Store Performance (Best Stores by Revenue)
store_performance = df_silver.groupBy(
    "store_id", "store_name"
).agg(
    _sum("total_amount").alias("store_revenue")
).orderBy(desc("store_revenue"))
display(store_performance)

# City-wise Revenue
city_revenue = df_silver.groupBy(
    "city"
).agg(
    _sum("total_amount").alias("city_revenue")
).orderBy(desc("city_revenue"))
display(city_revenue)

# New vs Returning Customers
from pyspark.sql.window import Window
from pyspark.sql.functions import min as _min, when

# Find first transaction date per customer
first_txn = df_silver.groupBy("customer_id").agg(_min("transaction_date").alias("first_txn_date"))
df_silver_with_first = df_silver.join(first_txn, "customer_id")
df_silver_with_first = df_silver_with_first.withColumn(
    "customer_type",
    when(df_silver_with_first["transaction_date"] == df_silver_with_first["first_txn_date"], "New").otherwise("Returning")
)
new_vs_returning = df_silver_with_first.groupBy("customer_type").agg(
    countDistinct("customer_id").alias("unique_customers"),
    _sum("total_amount").alias("total_revenue")
)
display(new_vs_returning)


# Save all KPI DataFrames to the gold layer in Delta format
total_sales_revenue_path = "abfss://demtest@demostorage31225.dfs.core.windows.net/gold/total_sales_revenue"
daily_sales_trends_path = "abfss://demtest@demostorage31225.dfs.core.windows.net/gold/daily_sales_trends"
monthly_sales_trends_path = "abfss://demtest@demostorage31225.dfs.core.windows.net/gold/monthly_sales_trends"
top_products_path = "abfss://demtest@demostorage31225.dfs.core.windows.net/gold/top_products"
top_categories_path = "abfss://demtest@demostorage31225.dfs.core.windows.net/gold/top_categories"
store_performance_path = "abfss://demtest@demostorage31225.dfs.core.windows.net/gold/store_performance"
city_revenue_path = "abfss://demtest@demostorage31225.dfs.core.windows.net/gold/city_revenue"
new_vs_returning_path = "abfss://demtest@demostorage31225.dfs.core.windows.net/gold/new_vs_returning"

total_sales_revenue.write.mode("overwrite").format("delta").save(total_sales_revenue_path)
daily_sales_trends.write.mode("overwrite").format("delta").save(daily_sales_trends_path)
monthly_sales_trends.write.mode("overwrite").format("delta").save(monthly_sales_trends_path)
top_products.write.mode("overwrite").format("delta").save(top_products_path)
top_categories.write.mode("overwrite").format("delta").save(top_categories_path)
store_performance.write.mode("overwrite").format("delta").save(store_performance_path)
city_revenue.write.mode("overwrite").format("delta").save(city_revenue_path)
new_vs_returning.write.mode("overwrite").format("delta").save(new_vs_returning_path)




store_id,store_name,product_id,product_name,year,month,monthly_sales,monthly_quantity,unique_customers
4,Downtown Mini Store,3,Yoga Mat,2024,11,2495.0,5,1
5,Mega Plaza,8,Desk Organizer,2024,7,1995.0,5,1
4,Downtown Mini Store,8,Desk Organizer,2024,7,1596.0,4,1
2,High Street Store,7,Smartwatch,2025,1,24995.0,5,1
3,Tech World Outlet,9,Dumbbell Set,2024,11,7996.0,4,1
3,Tech World Outlet,1,Wireless Mouse,2024,8,1599.98,2,1
1,City Mall Store,8,Desk Organizer,2024,11,399.0,1,1
3,Tech World Outlet,3,Yoga Mat,2025,5,1996.0,4,1
2,High Street Store,6,Water Bottle,2024,11,1196.0,4,1
3,Tech World Outlet,1,Wireless Mouse,2024,10,5599.93,7,2


customer_id,first_name,last_name,email,city,customer_lifetime_value,total_transactions
105,Riya,Singh,user105@example.com,Chennai,6643.95,3
130,Tanvi,Kapoor,user130@example.com,Pune,298.0,1
122,Tina,Kapoor,user122@example.com,Pune,3998.0,1
119,Kunal,Nair,user119@example.com,Bangalore,3998.0,1
116,Rakesh,Kapoor,user116@example.com,Kolkata,9484.470000000001,4
101,Ravi,Yadav,user101@example.com,Delhi,14997.0,1
126,Bhavna,Nair,user126@example.com,Mumbai,10596.0,2
107,Priya,Kapoor,user107@example.com,Ahmedabad,7996.0,1
103,Sonal,Sharma,user103@example.com,Bangalore,2399.9700000000003,1
125,Tarun,Verma,user125@example.com,Delhi,799.99,1


total_sales_revenue
134570.74


date,daily_sales,daily_quantity
2024-07-10,1995.0,5
2024-07-14,2395.99,5
2024-07-17,3999.95,5
2024-07-30,298.0,2
2024-08-11,1599.98,2
2024-08-27,6497.45,5
2024-09-05,2399.9700000000003,3
2024-09-21,598.0,2
2024-10-02,3999.95,5
2024-10-08,1599.98,2


year,month,monthly_sales,monthly_quantity
2024,7,8688.939999999999,17
2024,8,8097.43,7
2024,9,2997.9700000000003,5
2024,10,5998.93,8
2024,11,27083.0,17
2024,12,1995.0,5
2025,1,49990.0,10
2025,3,3740.0,10
2025,4,3998.0,2
2025,5,7985.47,16


product_id,product_name,product_revenue,total_quantity
7,Smartwatch,74985.0,15
9,Dumbbell Set,15992.0,8
1,Wireless Mouse,14399.819999999998,18
2,Bluetooth Speaker,10395.92,8
8,Desk Organizer,9975.0,25
3,Yoga Mat,4491.0,9
6,Water Bottle,3289.0,11
5,Notebook Set,1043.0,7


category,category_revenue
Electronics,99780.74
Fitness,23772.0
Accessories,9975.0
Stationery,1043.0


store_id,store_name,store_revenue
3,Tech World Outlet,42186.91
2,High Street Store,37333.92
5,Mega Plaza,29284.99
4,Downtown Mini Store,24072.920000000002
1,City Mall Store,1692.0


city,city_revenue
Mumbai,63381.979999999996
Delhi,17791.99
Kolkata,16577.920000000002
Ahmedabad,9595.98
Bangalore,8392.970000000001
Chennai,8138.95
Hyderabad,6394.95
Pune,4296.0


customer_type,unique_customers,total_revenue
Returning,6,47520.47
New,21,87050.26999999999
